# Data Cleaning

In [1]:
import pandas as pd # pandas is a library for data manipulation and analysis
import numpy as np # numpy is a library for numerical computations

## Load in the BLS data

This data is in a long format and contains many measures that we don't need for our analysis. After exploring the data, I have decided only to keep the TFP measure indexed to 2017 for two digit NAICS codes only.

In [2]:
# Read the Excel file into a pandas DataFrame
bls_data = pd.read_excel("../data/bls_data.xlsx", sheet_name="MachineReadable") 

In [3]:
# Pivoting the DataFrame into a wide format two separate columns for Measure and Units
pivoted = bls_data.pivot_table(index=["NAICS", "Industry", "Year"], columns=["Measure", "Units"], values="Value", aggfunc="first")

In [4]:
# Select only the TFP column with the unit "Index (2017=100)"
tfp_naics_long = pivoted.loc[:, 'Total factor productivity']["Index (2017=100)"].copy()

tfp_naics_long = tfp_naics_long.reset_index()
tfp_naics_long = tfp_naics_long.rename(columns={"Index (2017=100)": "TFP"})

tfp_naics_long["TFP_growth"] = tfp_naics_long.groupby("NAICS")["TFP"].pct_change()

In [5]:
# Shorten the DataFrame further to include only 2 digit NAICS codes.
tfp_naics_short = tfp_naics_long.loc[(tfp_naics_long["NAICS"].str.len() < 3) & ~(tfp_naics_long["NAICS"].isin(["DM", "ND"]))].reset_index(drop=True)

In [6]:
tfp_naics_short

,NAICS,Industry,Year,TFP,TFP_growth
0,11,"Agriculture, forestry, fishing, and hunting",1987,68.776,NaN
1,11,"Agriculture, forestry, fishing, and hunting",1988,63.369,-0.078618
2,11,"Agriculture, forestry, fishing, and hunting",1989,66.843,0.054822
3,11,"Agriculture, forestry, fishing, and hunting",1990,70.492,0.054591
4,11,"Agriculture, forestry, fishing, and hunting",1991,71.025,0.007561
...,...,...,...,...,...
624,MN,Manufacturing sector,2019,99.675,-0.016731
625,MN,Manufacturing sector,2020,98.769,-0.00909
626,MN,Manufacturing sector,2021,102.304,0.035791
627,MN,Manufacturing sector,2022,101.062,-0.01214


In [7]:
# Get unique pairs of NAICS and Industry
naics_industry = tfp_naics_short[["NAICS", "Industry"]].drop_duplicates().reset_index(drop=True)
naics_industry

,NAICS,Industry
0,11,"Agriculture, forestry, fishing, and hunting"
1,21,Mining
2,22,Utilities
3,23,Construction
4,42,Wholesale trade
5,51,Information
6,52,Finance and insurance
7,53,Real estate and rental and leasing
8,54,"Professional, scientific, and technical services"
9,55,Management of companies and enterprises


These NAICS codes represent the majority of the private sector economy in the US. MN represents the manufacturing NAICS codes 31-33.

## Load in the BDS data

This data is in a long format and contains only data at the 4 digit NAICS code granularity. In order to be consistent with the BLS data, I will convert it into the 2 digit NAICS code granularity and will also only keep the data for the same sectors and years as the BLS data.

In [8]:
bds_data = pd.read_csv("../data/bds_4naics.csv")

In [9]:
bds_data = bds_data.copy()

# Remove columns that contain "rate". This is done to clean the DataFrame and keep only the raw data.
# It is means that when grouping, we can simply use 'sum' as the aggregate function.
bds_data = bds_data.loc[:, ~bds_data.columns.str.contains("rate", case=False)]

# Create the 2 digit NAICS code column using the feature that the first two digits of a longer NAICS code are that industry's 2 digit sector.
bds_data["2naics"] = bds_data["vcnaics4"].astype(str).str[:2]

# Merge codes 31-33 into MN for consistency with the BLS data.
bds_data["2naics"] = bds_data["2naics"].replace({"31": "MN", "32": "MN", "33": "MN"})

In [10]:
# replace values containing "D" with np.nan
bds_data = bds_data.replace("D", value=np.nan)

# convert columns to numeric for proper functioning of groupby.
for col in bds_data.columns:
    if col not in ["Year", "2naics"]:
        try:
            bds_data[col] = pd.to_numeric(bds_data[col])
        except ValueError:
            print(f"Column {col} cannot be converted to int. It may contain non-integer values.")

In [11]:
# Group the dataframe by the 2 digit NAICS codes and the year and aggregate them using the sum function.
bds_2naics = bds_data.groupby(["2naics", "year"]).agg("sum").reset_index()

# remove vcnaics4 column and rename 2naics to NAICS
bds_2naics = bds_2naics.rename(columns={"2naics": "NAICS", "year": "Year"})
bds_2naics = bds_2naics.drop(columns=["vcnaics4"])

In [12]:
bds_2naics

,NAICS,Year,firms,estabs,emp,denom,estabs_entry,estabs_exit,job_creation,job_creation_births,job_creation_continuers,job_destruction,job_destruction_deaths,job_destruction_continuers,net_job_creation,firmdeath_firms,firmdeath_estabs,firmdeath_emp
0,11,1978,20370,21019,185795,178800,4011.0,3698.0,57357,20570.0,36787.0,43697,17668.0,26029.0,13660,2690.0,2715.0,12047.0
1,11,1979,20037,20633,185894,186550,3612.0,3650.0,50074,19437.0,30637.0,51578,20607.0,30971.0,-1504,2618.0,2662.0,13911.0
2,11,1980,19994,20586,187244,187525,3282.0,3392.0,45609,16313.0,29296.0,46396,15232.0,31164.0,-787,2581.0,2581.0,10799.0
3,11,1981,19235,19812,179027,182600,2810.0,3489.0,41572,14777.0,26795.0,48802,19376.0,29426.0,-7230,2486.0,2489.0,11514.0
4,11,1982,17973,18614,169727,172432,2600.0,3469.0,42786,16078.0,26708.0,48092,16623.0,31469.0,-5306,2415.0,2420.0,11377.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,MN,2018,237183,273785,11867119,11778528,18299.0,18584.0,1113717,240980.0,872737.0,939197,238605.0,700592.0,174520,13696.0,13769.0,124547.0
941,MN,2019,235305,272129,12064265,11960958,17161.0,18815.0,1085270,222948.0,862322.0,877348,237753.0,639595.0,207922,13804.0,13903.0,119545.0
942,MN,2020,231324,268369,11960402,12010626,16630.0,20458.0,977357,216658.0,760699.0,1079621,256545.0,823076.0,-102264,14375.0,14470.0,127688.0
943,MN,2021,227887,265398,11658587,11810038,18251.0,20646.0,988187,234806.0,753381.0,1289836,259129.0,1030707.0,-301649,13914.0,14046.0,114600.0


As there is not a firm births column, we must infer it from the number of firms each period and the number of firm deaths.

$$
\text{firms}_{i+1} = \text{firms}_{i} + \text{births}_{i} - \text{deaths}_{i}\\
\text{births}_{i} = \text{firms}_{i+1} - \text{firms}_{i} + \text{deaths}_{i}
$$

The job reallocation rate formula is given by:

$$
\text{job\_creation\_rate} + \text{job\_destruction\_rate} - ||\text{net\_job\_creation\_rate}||\\
$$

In [13]:
# Calculate firm births
bds_2naics["firmbirth_firms"] = bds_2naics["firms"].shift(-1) - bds_2naics["firms"] + bds_2naics["firmdeath_firms"]

# Calculate firm birth and death rates
bds_2naics["firm_birth_rate"] = bds_2naics["firmbirth_firms"] / bds_2naics["firms"]
bds_2naics["firm_death_rate"] = bds_2naics["firmdeath_firms"] / bds_2naics["firms"]

# Calculate reallocation rate
bds_2naics["job_reallocation_rate"] = (bds_2naics["job_creation"] + bds_2naics["job_destruction"] - abs(bds_2naics["job_creation"] - bds_2naics["job_destruction"])) / bds_2naics["denom"]

In [14]:
# Keep only columns of interest
bds_2naics = bds_2naics[["NAICS", "Year", "job_reallocation_rate", "firm_birth_rate", "firm_death_rate", "emp"]]

## Load in the output data

In order to calculate aggregates, I will weight them by sector output. This means we need output data at the same granularity as the BLS and BDS data, which I have got from the Bureau of Economic Analysis.

In [15]:
output_pre97 = pd.read_csv("../data/naics_output_pre97.csv")
output_post97 = pd.read_csv("../data/naics_output_post97.csv")

output_pre97.drop(columns=["Line"], inplace=True)
output_post97.drop(columns=["Line"], inplace=True)

output = pd.merge(output_pre97, output_post97, how="outer", on="Industry")

In [16]:
# Convert the data from wide to long format
output_data = output.melt(id_vars=["Industry"], var_name="Year", value_name="output")
output_data["Year"] = output_data["Year"].astype(int)
output_data["output_share"] = output_data["output"] / output_data.groupby("Year")["output"].transform("sum")

output_data = output_data.replace("Manufacturing", value="Manufacturing sector")

## Merging the data sets

Before we merge the data, we need to ensure that we have the same NAICS codes and years in both data sets.

In [17]:
# Remove additional NAICS industries that aren't common to both datasets. Remove early years that are not in the BLS data.
bds_2naics = bds_2naics[bds_2naics["NAICS"].isin(tfp_naics_short["NAICS"])].reset_index(drop=True)
bds_2naics = bds_2naics[bds_2naics["Year"].isin(tfp_naics_short["Year"])].reset_index(drop=True)

# Removes the additional year that is in the BLS data but not in the BDS data.
tfp_naics_short = tfp_naics_short[tfp_naics_short["Year"].isin(bds_2naics["Year"])].reset_index(drop=True)

In [18]:
# Check the dimensions of each data set match (only the rows need to).
print(f"BDS data dimensions: {bds_2naics.shape}")
print(f"BLS data dimensions: {tfp_naics_short.shape}")

BDS data dimensions: (612, 6)
BLS data dimensions: (612, 5)


In [19]:
# Merge bds_2naics with tfp_naics_short on Year and NAICS.
merged_data = pd.merge(tfp_naics_short, bds_2naics, on=["Year", "NAICS"], how="inner")
merged_data = pd.merge(merged_data, output_data, on=["Industry", "Year"], how="inner")

In [20]:
merged_data.to_csv("../data/merged.csv", index=False)